In [ ]:
from covid19.models import SEIRBayes
from covid19.estimation import ReproductionNumber
from covid19.data import load_cases, load_population

from joblib import Parallel, delayed
from tbats import TBATS

import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd


CPU_COUNT = multiprocessing.cpu_count()
parallel = Parallel(n_jobs=CPU_COUNT, verbose=3)

SAMPLE_SIZE = 10_000
DEFAULT_SEIR_PARAMS = dict(
    gamma_inv_dist=(7, 13, 0.95, "lognorm"),
    alpha_inv_dist=(4, 7, 0.95, "lognorm"),
    fator_subr=20 # subnot / (1-assint)
)
one_day = pd.Timedelta('1D')

plt.style.use('seaborn-darkgrid')
plt.rcParams['image.cmap'] = 'Dark2'
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = (12, 5)

In [ ]:
def estimate_active_cases(cases, gamma_sup):
    return (
        cases
        .assign(
            estimatedActiveCases=lambda x: (
                x["newCases"]
                .rolling(window=int(gamma_sup), min_periods=1)
                .sum()
            )
        )
        .assign(
            estimatedRemovedCases=lambda x: (
                x["totalCases"] - x["estimatedActiveCases"]
            )
        )
    )


def estimate_r0_mean(cases):
    incidence = (cases['newCases']
                 .rename('incidence')
                 .reset_index()
                 .rename(columns={'date': 'dates'}))
    dynamic_window_width = min(len(incidence) - 2, 14)
    Rt = ReproductionNumber(
        incidence=incidence,
        prior_shape=5.12,
        prior_scale=0.64,
        si_pars={"mean": 4.89, "sd": 1.48},
        window_width=dynamic_window_width,
    )
    Rt.compute_posterior_parameters()
    rt_samples = Rt.sample_from_posterior(sample_size=SAMPLE_SIZE)
    return rt_samples[:, -1].mean()


def mse(pred, true):
    return np.sqrt((true - pred)
                   .pipe(np.power, 2)
                   .sum())


def mape(pred, true):
    return ((true - pred)
            .abs()
            .sum())/true.sum()


def predict_tbats(y_train, len_test_period):
    estimator = TBATS(
        seasonal_periods=None,
        use_arma_errors=None,
        use_box_cox=None,
        use_trend=None,
        use_damped_trend=None,
        show_warnings=False,
    )
    fitted_model = estimator.fit(y_train)
    return fitted_model.forecast(steps=len_test_period)


def run_and_plot_single_comparison_with_tbats(*args, **kwargs):
    r = run_single(*args, **kwargs, return_series=True)
    r_tbats = run_single(*args, **kwargs, return_series=True, use_tbats=True)
    fig, ax = plt.subplots(figsize=(15,7.5))
    (r['df']
     [['totalCases', 'AvgtotalCases_pred']]
     .rename(columns={'totalCases': 'Histórico', 
                      'AvgtotalCases_pred': 'SEIR-Bayes'})
     .plot
     .line(ax=ax,
           color = ['b', 'y'],
           style=['-','--']))
    (r_tbats['df']
     ['AvgtotalCases_pred']
     .rename('TBATS')
     .plot
     .line(ax=ax,
           color='r',
           style='--'))
    ax.legend(loc='upper left')
    ax.fill_between(r['df'].index,
                    r['df']['AvgtotalCases_pred'] - 3*r['df']['StdtotalCases_pred'],
                    r['df']['AvgtotalCases_pred'] + 3*r['df']['StdtotalCases_pred'],
                    color='y',
                    alpha=0.3)
    plt.title(f"{kwargs['place']} @ t_max={kwargs['t_max']}, date={kwargs['date']}")
    plt.show()
    r.pop('df')
    r_tbats.pop('df')
    return pd.DataFrame([r, r_tbats])

# Load data

In [ ]:
cases = pd.concat([load_cases('state'), load_cases('city')], axis=1)
cases.head()

In [ ]:
population = pd.concat([load_population('state'), load_population('city')], axis=0)
population.head()

# Generate train and test periods

In [ ]:
start_date = (
    (cases.loc(axis=1)[:, 'newCases'] > 100)
    .loc(axis=1)[lambda df: df.any()]
    .droplevel(1, axis=1)
    .apply(lambda s: s.index[s.argmax()])
    .rename('start_date')
)

start_date

In [ ]:
def split_dates(stride, start_date=start_date): 
    return (
        start_date
        .apply(lambda d: pd.date_range(d, 'now', freq=stride)[1:])
        .to_dict()
    )

split_dates('15D')['SP']

# Augment Cases dataframe with estimatedActiveCases

In [ ]:
gamma_window = DEFAULT_SEIR_PARAMS['gamma_inv_dist'][1]

cases_aug = pd.concat(
    [(estimate_active_cases(cases[place], gamma_window)
    .assign(place=place)
    .set_index('place', append=True)
    .unstack('place')
    .swaplevel(0, 1, axis=1))
 for place in start_date.index], axis=1)

cases_aug.head()

# Run train-test splits

In [ ]:
def run_single(**params):
    try:
        place, date, t_max = params['place'], params['date'], int(params['t_max'])
        return_series = params.get('return_series', False)
        use_tbats = params.get('use_tbats', False)
        
        date = pd.Timestamp(date)
        N = population[place]
        cases_aug_place = cases_aug[place]

        train_start_date = start_date[place]
        train_end_date = date
        test_start_date = date + one_day
        test_end_date = np.min([date + pd.Timedelta(f'{t_max}D'),
                                cases_aug_place.index.max()])

        train_period = f'{train_start_date.date()}:{train_end_date.date()}'
        test_period = f'{test_start_date.date()}:{test_end_date.date()}'

        cases_aug_train = cases_aug_place.loc[train_start_date:train_end_date]
        cases_aug_test = cases_aug_place.loc[test_start_date:test_end_date]

        n_test_points = int(cases_aug_test.shape[0])
        n_train_points = int(cases_aug_train.shape[0])
        
        common_output = {
            'place': place,
            'train_period': train_period,
            'test_period': test_period,
            'n_test_points': n_test_points,
            'n_train_points': n_train_points,
            'model': 'TBATS' if use_tbats else 'SEIR-Bayes',
            'date': date.date(),
            't_max': t_max,
        }
        
        if n_test_points < 7:
            raise Exception('Not enough test points '
                            f'({n_test_points}) @ {place} and {date.date()}')
            
        assert not cases_aug_test.index.isin(cases_aug_train.index).any()
        assert all(cases_aug[train_start_date:test_end_date].index
                   == cases_aug_train.index.tolist() + cases_aug_test.index.tolist())

        if use_tbats:
            avg_total_cases_pred = predict_tbats(cases_aug_train['totalCases'], t_max)
            std_total_cases_pred = np.zeros_like(avg_total_cases_pred)
        else:
            I0 = cases_aug_train["estimatedActiveCases"].iloc[-1]
            r0 = estimate_r0_mean(cases_aug_train)
            E0 = r0*I0
            R0 = cases_aug_train["estimatedRemovedCases"].iloc[-1]

            model = CompartmentalModel(
                NEIR0=(N, E0, I0, R0),
                r0_dist=np.full(SAMPLE_SIZE, r0),
                **DEFAULT_SEIR_PARAMS,
                t_max=t_max,
            )
            _, _, I, R, _, _ = model.sample(SAMPLE_SIZE)
            I /= DEFAULT_SEIR_PARAMS['fator_subr']
            R /= DEFAULT_SEIR_PARAMS['fator_subr']
            avg_total_cases_pred = (I + R).mean(axis=1)
            std_total_cases_pred = (I + R).std(axis=1)

        if return_series:
            avg_total_cases_pred = pd.Series(avg_total_cases_pred,
                                             index=pd.date_range(test_start_date, periods=t_max),
                                             name='AvgtotalCases_pred')
            std_total_cases_pred = pd.Series(std_total_cases_pred,
                                             index=pd.date_range(test_start_date, periods=t_max),
                                             name='StdtotalCases_pred')
            cases_aug_place = pd.concat([avg_total_cases_pred, 
                                         std_total_cases_pred,
                                         cases_aug_place], axis=1)
        
        return {
            **common_output,
            'mape': mape(avg_total_cases_pred[:n_test_points], cases_aug_test['totalCases']),
            'mse': mse(avg_total_cases_pred[:n_test_points], cases_aug_test['totalCases']),
            'df': cases_aug_place if return_series else None,
        }

    except Exception as e:
        return {
            **common_output,
            'exception': e,
            'df': None
        }

# aqui vocês podem adicionar mais casos de teste e rodar em paralelo
metrics = parallel([delayed(run_single)(place=place,
                                        date=date,
                                        t_max=t_max,
                                        use_tbats=use_tbats)
                    for use_tbats in [True, False]
                    for place in ['MG', 'São Paulo/SP']
                    for date in split_dates(stride='15D')[place]
                    for t_max in [15, 60]])
metrics = pd.DataFrame(metrics)
metrics.head()

# Analysis

In [ ]:
1 - metrics['exception'].isna().mean()

In [ ]:
(metrics
 .query("exception.isna()")
 ['mape']
 .describe())

In [ ]:
metrics.query("exception.isna()").corr()

In [ ]:
(metrics
 .query("exception.isna()")
 .sort_values('mape')
 .drop_duplicates(subset=['place'])
 .nsmallest(20, 'mape'))

In [ ]:
run_and_plot_single_comparison_with_tbats(place='MG', t_max=30, date='2020-05-24')

In [ ]:
(metrics
 .groupby(['n_train_points', 'model'])
 ['mape']
 .mean()
 .unstack('model')
 .plot(style='o--', title='Average MAPE'));

In [ ]:
(metrics
 .groupby(['n_test_points', 'model'])
 ['mape']
 .mean()
 .unstack('model')
 .plot(style='o--', title='Average MAPE'));

In [ ]:
(metrics
 .groupby(['model', 't_max'])
 ['mape']
 .mean()
 .unstack('model')
 .plot(style='o--', title='Average MAPE'));

In [ ]:
(metrics
 .groupby([metrics['n_test_points'] // 5, 'model'])
 ['mape']
 .mean()
 .unstack('model')
 .plot(style='o--', title='Average MAPE'))

plt.xlabel('n_test_points // 5');